# Standardizing Mass Spectrometry Data to Calculate Periplasmic Protein Density

In [1]:
import numpy as np
import pandas as pd 
import size.viz # Custom module for specifying plotting colors
import size.analytical
import altair as alt
import scipy.stats
import scipy.optimize
colors, palette = size.viz.altair_style()

This notebook covers my approach of exploring mass spectrometry data to see how periplasmic protein content and density scale with growth rate.

## Brief Overview of Datasets
Most mass spectrometry measurements are not in absolute units, meaning they are not in proper "per cell" measurements. I recently tried to assemble a list of datasets that are trustworthy in their reporting, but I quickly discovered that there's a pretty wide breadth in how people define "per cell" and the controls that are needed to ensure it is correct. In general, I only really believe relative abundance measurements. 

Luckily, we know quite a bit about basic aspects of *E. coli* physiology, meaning we know some aspects of the typical protein density, have empirical relations for how cell volume scales with growth rate, and have annotation of nearly the entire proteome at a functional and localization level. Using the relative abundance measurements and our knowledge of the molecular weight of individual proteins, we can pretty easily compute a mass fraction of the proteome occupied by any single protein. Using these aforementioned empirical measurements, we can calculate our way to "per cell" measurements from relative measurements in mass spectrometry. 

## The Procedure
I've collected a set of mass spectrometry measurements and have standardized their gene names such that they are directly comparable. The sources of these measurements are as follows. 


* Schmidt *et al.* "The Quantitative and Condition-Dependent *Escherichia coli* Proteome." Nat Biotechnol 2016, 34 (1), 104–110. https://doi.org/10.1038/nbt.3418.


* Valgepea, K.; Adamberg, K.; Seiman, A.; Vilu, R. *Escherichia coli* Achieves Faster Growth by Increasing Catalytic and Translation Rates of Proteins. Mol. BioSyst. 2013, 9 (9), 2344–2358. https://doi.org/10.1039/C3MB70119K.


* Peebo *et al.*. "Proteome Reallocation in Escherichia coli with Increasing Specific Growth Rate". Molecular BioSystems 2015, 11 (4), 1184–1193. https://doi.org/10.1039/C4MB00721B.


* Mori *et al.* "From Coarse to Fine: The Absolute *Escherichia coli* Proteome under Diverse Growth Conditions." Mol Syst Biol 2021, 17 (5). https://doi.org/10.15252/msb.20209536. **Note that I only used some of the data from this work as I wanted to avoid N- and R-limitation as well as osmotic stress**


* Caglar *et al.*. "The *E. coli* Molecular Phenotype under Different Growth Conditions." Sci Rep 2017, 7 (1), 1–15. https://doi.org/10.1038/srep45303 **I only considered the C-limitation conditions**


* Soufi *et al.*  "Characterization of the *E. coli* Proteome and Its Modifications during Growth and Ethanol Stress." Frontiers in Microbiology 2015, 6. https://doi.org/10.3389/fmicb.2015.00103. **I did not use the Ethanol stress samples.**


People are generally terrible at providing their data in an actually useable form, so it took a lot of work to get this into a standardized format. I've saved these data in a single, long-form tidy dataframe in this repository (`/data/source/compiled_mass_fractions.csv`). 

To get to a standardized "per cell" resolution, we have to know details of how much protein there is per cell. This can be a bit difficult to directly measure with any appreciable level of certainty, so one approach is to use our empirical knowledge of the total *protein density* of a cell and how the cell *volume* changes with growth rate. For the former, Basan et al. 2015 did a careful measurement of total protein per OD and cells per OD as a function of growth rate. They also measured the cell volume, though that seems to be systematically over estimated. Si et al 2017 and 2019 did very careful measurements of cell dimensions and total cell volume across growth rates and for multiple strain backgrounds. We can use both of these data sets to figure out what the protein density is per cell.

## Empirical Fitting for Cell Length, Width, and Volume
We will start with figuring out some empirical relations to describe how the total cell volume changes as a function of growth rate for 
two strain background -- NCM3722 and MG1655. To begin, we will load and view the data.

In [2]:
# Load the cell volume data
vol_data = pd.read_csv('../../data/source/Si2017/Si2017_c_limitation.csv')

# Set up the plotting base
vol_plot = alt.Chart(
                data=vol_data,
                width=300,
                height=300,
                ).mark_point(
                    size=100,
                    opacity=0.75
                ).encode(
                    x=alt.X('growth_rate_hr:Q'),
                    y=alt.Y(alt.repeat("column"), type='quantitative'),
                    color=alt.Color('strain:N')
                ).repeat(
                    column=['cell_length_um', 'cell_width_um', 'cell_volume_fL']
                ).properties(
                    title='Si et al., 2017 - cell dimension measurements'
                )
vol_plot

alt.RepeatChart(...)

There are clear differences between MG1655 and NCM3722, which was kind of expected. As *E. coli* cells grow exponentially, we can do an empirical fit of an exponential with the form
$$
V_{cell} = ae^{b \lambda} + c, \tag{1}
$$
Where $a$, $b$, and $c$ are constants and $\lambda$ is the growth rate.

In [4]:
# Define an exponential function to fit the cell volume
def exp_fit(lam, a, b):
    return a * np.exp(lam * b)

lam_range = np.linspace(0, 2.5, 100)
fit_dfs = []
for g, d in vol_data.groupby(['strain']):
    # Perform the fit
    popt = scipy.optimize.curve_fit(exp_fit,
                                d['growth_rate_hr'].values, 
                                d['cell_volume_fL'].values)

    # Unpack and print the parameters
    a, b = popt[0]
    print(f"""
    Best-fit parameters for {g} cell volume
    ============================================
    a = {a:0.3f} µm^3
    b = {b:0.3f} hr
    """)

    # Compute the fit and create a dataframe
    fit_df = pd.DataFrame([])
    fit_df['growth_rate_hr'] = lam_range
    fit_df['cell_volume_fL'] = a * np.exp(b * lam_range)
    fit_df['strain'] = g
    fit_dfs.append(fit_df)
fit_df = pd.concat(fit_dfs, sort=False)


    Best-fit parameters for MG1655 cell volume
    a = 0.526 µm^3
    b = 1.059 hr
    

    Best-fit parameters for NCM3722 cell volume
    a = 0.246 µm^3
    b = 1.319 hr
    


Let's see how good this fit is by plotting it. 

In [7]:
data_plot = alt.Chart(
            width=600,
            height=600,
            data =vol_data
            ).mark_point(
                size=100, 
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y=alt.Y('cell_volume_fL:Q', scale=alt.Scale(type='log')),
                color='strain:N'
            )

fit_plot = alt.Chart(
            data=fit_df
            ).mark_line( 
                size=1
            ).encode(
                x='growth_rate_hr:Q',
                y='cell_volume_fL:Q',
                color='strain:N'
            )

data_plot + fit_plot

alt.LayerChart(...)

This seems to do a pretty good job at an empirical level. It's again important to note that there is a big difference between NCM3722 and MG1655.
We should do the same procedure to get an empirical description of how the cell length and width scale with growth rate as we will use 
these quantities to compute the envelope density. 

In [8]:
# Perform a similar fitting procedure for cell width and length
dim_dfs = []
for g, d in vol_data.groupby(['strain']):

    # Do the fits
    length_popt = scipy.optimize.curve_fit(exp_fit, 
                                          d['growth_rate_hr'].values, 
                                          d['cell_length_um'].values)
    width_popt = scipy.optimize.curve_fit(exp_fit, 
                                          d['growth_rate_hr'].values, 
                                          d['cell_width_um'].values,
                                          maxfev=100000)
    # unpack and compute the fits
    length_a, length_b, length_c = length_popt[0]
    length_fit = length_a * np.exp(length_b * lam_range) + length_c
    width_a, width_b, width_c = width_popt[0]
    width_fit = width_a * np.exp(width_b * lam_range) + width_c

    # Assemble the dataframe
    _df = pd.DataFrame([])
    _df['growth_rate_hr'] = lam_range
    _df['cell_length_um'] = length_fit
    _df['cell_width_um'] = width_fit
    _df['strain'] = g
    dim_dfs.append(_df)

    # Print out the results
    print(f"""
    Cell Dimension fits for {g} 
    ==================================

    Width 
    ------
    a = {width_a :0.3f} µm
    b = {width_b: 0.4f} hr
    c = {width_c:0.3f} µm

    Length 
    ------
    a = {length_a:0.3f} µm
    b = {length_b:0.3f}  hr
    c = {length_c:0.3f} µm
    """)

dim_df = pd.concat(dim_dfs)

ValueError: not enough values to unpack (expected 3, got 2)

In [6]:
# Generate plots to see how it looks
vol_data['type'] = 'experiment'
dim_df['type'] = 'fit'

charts = []
for dim in ['cell_length_um', 'cell_width_um']:
    dim_point = alt.Chart(
            data=vol_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y=dim + ':Q',
                color='strain:N')
    dim_fit = alt.Chart(
            data=dim_df,
            ).mark_line(
                size=1
            ).encode(
                x='growth_rate_hr:Q',
                y=dim + ':Q',
                color='strain:N'
            )
    charts.append(dim_point + dim_fit)

alt.hconcat(charts[0],charts[1])

alt.HConcatChart(...)

These fits also look okay, though there was some trouble fitting the width from MG1655. Nevertheless, to use these we can set up functions to compute the dimensions froma supplied growth rate. 

In [18]:
def empirical_cell_volume(lam, strain='MG1655'):
    """
    Computes the cell volume at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    vol : positive float
        The cell volume in units of fL (same as cubic microns)
    """

    if strain == 'MG1655':
        a = 0.539 #0.816
        b = 1.056 #0.869
        c = 0 #-0.404
    elif strain == 'NCM3722':
        a = 0.236
        b = 1.338
        c = 0.023
    vol = a * np.exp(b * lam) + c
    return vol

def empirical_cell_length(lam, strain='MG1655'):
    """ 
    Computes the cell length at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    length : positive float
        The cell length in units of micron
    """

    if strain == 'MG1655':
        a = 1.036
        b = 0.798
        c = 0.938
    elif strain == 'NCM3722':
        a = 0.470
        b = 0.852
        c = 1.642
    length = a * np.exp(b * lam) + c
    return length


def empirical_cell_width(lam, strain='MG1655'):
    """ 
    Computes the cell width at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    width : positive float
        The cell width in units of micron
    """

    if strain == 'MG1655':
        a = 642.068 
        b = 0.003 
        c = -641.434
    elif strain == 'NCM3722':
        a = 0.394
        b = 0.474
        c = 0.061 
    length = a * np.exp(b * lam) + c
    return length


## Estimation of Protein Density Scaling

With functions in hand that do a pretty good job of empirically determining the cell length, width, and volume as a function of growth rate, we now need an estimate of what the protein density per unit cell volume. As mentioned earlier, Basan *et al.* did a careful measurement of the amount of protein biomass per OD ml and the number of cells per OD ml, as shown below.

In [10]:
# Load the calibration data
cal_data = pd.read_csv('../../data/source/Basan2015/Basan2015_calibration_curve.csv')
cal_data['cells_per_OD_1E8'] = cal_data['cells_per_OD'].values / 1E8
# Plot the protein biomass and cell number as a function of growth rate. 
cal_plot = alt.Chart(
            data=cal_data,
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y=alt.Y(alt.repeat('column'), type='quantitative')
            ).repeat(
                column=['ug_protein_per_OD', 'cells_per_OD_1E8']
            )
cal_plot

alt.RepeatChart(...)

These data make intuitive sense -- in general, the protein density you get per OD is roughly constant whereas the total number of cells per OD unit decreases as growth rate increases (meaning as cells get larger). We can compute the total mass of a cell as a function of growth rate as the ratio of these two quantities.

In [11]:
cal_data['fg_protein_per_cell'] = 1E9 * cal_data['ug_protein_per_OD'].values / cal_data['cells_per_OD'].values
cal_plot = alt.Chart(
            data=cal_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y='fg_protein_per_cell:Q'
            )
cal_plot

alt.Chart(...)

For our case, we need to be able to compute the protein density *per unit of cell volume*, not per cell. While Basan *et. al.* report their cell volumes, they seem to be consistenly larger than expected. We can use our empirical descriptions of cell volume as a function of growth rate to get an estimate of the "correct" volume at each growth rate. 

In [33]:
# Calculate the corrected cell volume for NCM
cal_data['corrected_volume_fL'] = empirical_cell_volume(cal_data['growth_rate_hr'], strain='NCM3722')
cal_plot = alt.Chart(
            data=cal_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='corrected_volume_fL:Q',
                y='fg_protein_per_cell:Q'
            )
cal_plot

alt.Chart(...)

This looks like an approximately linear relationship between cell size and cell volume, which is would be as expected if the density was approximately constant. To that end, we can 
do a simple linear regression to get the slope and intercept, and therefore reliably compute the protein mass per cell as a function of cell volume. 

In [34]:
# Perform a simple linear regression
popt = scipy.stats.linregress(cal_data['corrected_volume_fL'], cal_data['fg_protein_per_cell'])

# Unpack parameters report the results
density  = popt[0]
offset = popt[1]

print(f"""
Estimated protein density from Basan et al. 2015
=================================================
density, ρ = {density:0.3f} fg protein / fL
offset, c = {offset:0.3f} fg protein / cell
""")

# Compute the fits for display
vol_range = np.linspace(0, 3.5, 100)
fit = density * vol_range + offset
fit_df = pd.DataFrame([])
fit_df['volume'] = vol_range
fit_df['protein_per_cell'] = fit

# Set the canvases for plotting
cal_points = alt.Chart(
        data=cal_data
        ).mark_point(
            size=100,
            opacity=0.75
        ).encode(
            x=alt.X('corrected_volume_fL:Q', title='cell volume [fL]'),
            y=alt.Y('fg_protein_per_cell:Q', title='fg protein / fL'),
        )   

cal_fit = alt.Chart(
        data=fit_df
        ).mark_line(
            size=1
        ).encode(
            x=alt.X('volume:Q', title='cell volume [fL]'),
            y=alt.Y('protein_per_cell:Q', title='fg protein / fL')
        )
cal_points + cal_fit


Estimated protein density from Basan et al. 2015
density, ρ = 320.843 fg protein / fL
offset, c = 34.662 fg protein / cell



alt.LayerChart(...)

This also looks reasonable, though more data would always be better for getting a sense of uncertainty. Assuming that the protein density is similar between NCM3722 and MG1655 (which I think is reasonable), we can define a function that computes the fg protein per cell as a function of the cell volume, which we can compute as a function of the growth rate!

In [14]:
def compute_protein_per_cell(lam, strain='MG1655'):
    """
    Given calibration curve and empirical relations, compute the total mas of
    of protein per cell as a function of the growth rate. 

    Parameters 
    -----------
    lam : positive float 
        The growth rate (in inverse hours) at which you want to compute the 
        protein mass 
    strain : string, 'NCM3722' or 'MG1655'
        The E. coli strain used in determination of cell volume. Default is
        MG1655. 

    Returns
    --------
    prot : positive float
        The total mass of protein per cell in units of femtograms. 
    """

    # Determine the cell volume
    vol = empirical_cell_volume(lam, strain)

    # Compute and return the protein 
    prot = 34.662 + 320.843 * vol
    return prot

## Converting From Mass Fraction to Total Mass
To make use of the proteomic data, we have to be able to compute the mass of each protein and/or protein class. As of now, we only have the mass fractions. The total 
mass per cell of each protein can be computed simply as the product of the mass fraction and the total protein mass, which we can now get from the growth rate
given the above analysis. We can load the data and compute the total mass per cell 
of each protein


In [19]:
# Load the proteomics data 
data = pd.read_csv('../../data/compiled_mass_fractions.csv')

# Standardize some of the strains to either NC or MG
data.loc[data['strain'].isin(['EQ 59', 'EQ353', 'NQ1243', 'NQ1390']), 
        'strain'] = 'NCM3722'
data.loc[data['strain'].isin(['REL606', 'MG1655', 'BW25113']),
        'strain'] = 'MG1655'

# group by strain and compute the total protein mass
data.loc[data['strain']=='NCM3722', 
         'fg_protein_per_cell'] = compute_protein_per_cell(
                 data[data['strain']=='NCM3722']['growth_rate_hr'], 
                 strain='MG1655')
data.loc[data['strain']=='MG1655', 
        'fg_protein_per_cell'] = compute_protein_per_cell(
                data[data['strain']=='MG1655']['growth_rate_hr'], 
                strain='MG1655')
data.head()

,gene_name,mass_frac,condition,strain,growth_rate_hr,dataset_name,go_terms,cog_class,cog_letter,fg_protein_per_cell
0,aas,0.000061,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016874; GO:0016746; GO:0016740; GO:0016020...,metabolism,Q,328.210357
1,accA,0.000429,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016743; GO:0016740; GO:0042802; GO:0009317...,metabolism,I,328.210357
2,accB,0.000578,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0009305; GO:0005829; GO:0005515; GO:0006631...,metabolism,H,328.210357
3,accC,0.000782,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0005515; GO:0016874; GO:0006631; GO:0006629...,metabolism,I,328.210357
4,accD,0.000496,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016743; GO:0016740; GO:0009317; GO:0006631...,metabolism,I,328.210357


This is good if we care about single proteins, but we care about entire classes of proteins, namely those in the periplasm. There are a few different classification schemes we could do. The first one I'll look at is the classification based on GO terms, which are well supported in the E. coli databases. 


Specifically, we can filter the above datatable by looking *only* at those proteins associated with the GO term  `GO:0042597` or its synonym `GO:0005620`, which is the 
compartment "periplasmic space". We can filter our mass spec dataframe by these 
GO terms (and dataset/growth rate) and sum all the proteins to get the total 
periplasmic protein mass. 

In [24]:
# Compute the per protein mass
data['fg_per_cell'] = data['mass_frac'] * data['fg_protein_per_cell']

# Drop NaNs
data.dropna(inplace=True)

# Filter by go terms
periplasm = data.loc[(data['go_terms'].str.contains('GO:0042597')) |
                 (data['go_terms'].str.contains('GO:0005620'))]

# Group by the identifiers and sum to get the total periplasmic protein mass
periplasm_grouped  = periplasm.groupby(['strain', 'dataset_name', 
                    'growth_rate_hr']).sum().reset_index()
periplasm_grouped = periplasm_grouped[['strain', 'dataset_name', 'growth_rate_hr', 'mass_frac', 'fg_per_cell']]

# Plot the result
tot_periplasm_mass = alt.Chart(
        data=periplasm_grouped[periplasm_grouped['fg_per_cell'] <= 50], 
        ).mark_point(
            size=100,
            opacity=0.75
        ).encode(
            x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'),
            y=alt.Y('fg_per_cell:Q', title='total fg periplasmic protein per cell'),

            color=alt.Color('dataset_name:N')
        ).properties(
            title='total periplasmic protein (GO:0042597 + GO:0005620)'
        )

tot_periplasm_mass

alt.Chart(...)

This data is in line with what we've been discussing over the past few weeks -- it appears that there's a really constant total mass of periplasmic protein that is expressed, regardless of the growth rate. Even at very fast growth rates, the mass is only a factor of ≈ 1.5 times higher than in the slowest growth rate.  

From this plot, it also seems like there's pretty broad agreement between the datasets,
even considering there are different strains. To see directly how comparable they are (without needing to do any conversion), let's just examine the mass fractions.

In [15]:
mass_fracs = alt.Chart(
        data=periplasm_grouped,
        ).mark_point(
            size=100, 
            opacity=0.75
        ).encode(
            x=alt.X('growth_rate_hr', title='growth rate [per hr]'),
            y=alt.Y('mass_frac:Q', title='periplasmic protein mass fraction'),
            color=alt.Color('dataset_name:N', title='source')
        )
mass_fracs

alt.Chart(...)

This is also in agreement with what we've been finding. There's a consistent 
inverse correlation between the periplasmic mass fraction and the growth rate, though it is relaively mild. There does appear to be a minimum mass mfraction of a few percent, which is met at the faster growth rates.

## Calculating Periplasmic Protein Density
To calculate the periplasmic protein density, we have to know i) the mass of periplasmic protein in the cell and ii) the total volume of the periplasmic space. We can assume that the width of the periplasm (meaning, the size of the space between the inner and outer membrane) is around δ ≈ 25 nm. To compute the volume, we can compute
the difference between two spherocylinders where the length and width of one of them 
is either larger or smaller by δ, respectively. I've written a function to do this in the `size.analytical` module, which I'll just call here for brevity.

In [16]:
# Use my module to calculate the envelope volume
delta = 0.025

# Compute the empirical cell width and length
for s in ['MG1655', 'NCM3722']:
    periplasm_grouped.loc[periplasm_grouped['strain']==s, 'cell_length_um'] = empirical_cell_length(periplasm_grouped[periplasm_grouped['strain']==s]['growth_rate_hr'].values, strain=s) 
    periplasm_grouped.loc[periplasm_grouped['strain']==s, 'cell_width_um'] = empirical_cell_width(periplasm_grouped[periplasm_grouped['strain']==s]['growth_rate_hr'].values, strain=s) 

# Compute the envelope volume
periplasm_grouped['periplasmic_volume'] = size.analytical.envelope_volume(periplasm_grouped['cell_length_um'], periplasm_grouped['cell_width_um'], delta)

# Compute the periplasmic protein density
periplasm_grouped['periplasmic_protein_density'] = periplasm_grouped['fg_per_cell'].values / periplasm_grouped['periplasmic_volume'].values


# Generate a plot
density_plot = alt.Chart(
            data=periplasm_grouped
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'),
                y=alt.Y('periplasmic_protein_density:Q', title='periplasmic protein density [fg / fL]'),
                color=alt.Color('dataset_name:N')
            )
density_plot

alt.Chart(...)

There's a definite and super interesting correlation between the density and 
growth rate, which makes a lot of sense considering the above results. There are 
still a few weirdos from the Mori data. We can adjust the y scaling to get a better
idea of the actual behavior.

In [17]:
# Generate a plot
density_plot = alt.Chart(
            data=periplasm_grouped
            ).mark_point(
                size=100,
                opacity=0.75,
                clip=True
            ).encode(
                x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'),
                y=alt.Y('periplasmic_protein_density:Q', title='periplasmic protein density [fg / fL]',
                        scale=alt.Scale(domain=[0, 200])),
                color=alt.Color('dataset_name:N')
            )
density_plot

alt.Chart(...)

Wow, so this looks like there may be two distinct populations with all of the NCM data (Mori *et al.*) systematically resting above the other data, which is all MG1655, REL606, or BW25113. It would be good to see how Roshali's data aligns with this.

In [21]:
cal_data['density'] = cal_data['fg_protein_per_cell'].values / cal_data['reported_volume_fL'].values

KeyError: 'reported_volume_fL'

In [20]:
alt.Chart(cal_data).mark_point(size=100).encode(
    x=alt.X('growth_rate_hr:Q'),
    y=alt.Y('density:Q')
)

alt.Chart(...)

In [30]:
_grouped = data.groupby(['dataset_name', 'condition', 'growth_rate_hr']).sum().reset_index()
alt.Chart(_grouped).mark_point(size=100, opacity=0.75).encode(x='growth_rate_hr:Q', y='fg_per_cell:Q', color='dataset_name:N')

alt.Chart(...)

In [36]:
_grouped['vol'] = empirical_cell_volume(_grouped['growth_rate_hr'])

In [37]:
_grouped

,dataset_name,condition,growth_rate_hr,mass_frac,fg_protein_per_cell,fg_per_cell,vol
0,Caglar et al. 2017,gluconate_growth,0.663012,1.0,2.405742e+06,382.957948,1.085565
1,Caglar et al. 2017,glucose_time_course,0.774755,1.0,1.883779e+06,426.580467,1.221527
2,Li et al. 2014,MOPS complete,1.934364,1.0,4.155264e+06,1368.213310,4.156398
3,Li et al. 2014,MOPS complete without methionine,1.569390,1.0,2.106590e+06,941.703199,2.827056
4,Li et al. 2014,MOPS minimal,0.738700,1.0,1.383291e+06,411.939047,1.175893
...,...,...,...,...,...,...,...
66,Valgepea et al. 2013,glucose_minimal,0.110000,1.0,3.044339e+05,228.897667,0.605392
67,Valgepea et al. 2013,glucose_minimal,0.210000,1.0,2.946246e+05,250.531099,0.672818
68,Valgepea et al. 2013,glucose_minimal,0.310000,1.0,3.237227e+05,274.574002,0.747755
69,Valgepea et al. 2013,glucose_minimal,0.400000,1.0,3.507303e+05,298.493911,0.822308


In [44]:
alt.Chart(_grouped).mark_point(size=100, opacity=0.75).encode(
            y=alt.Y('fg_per_cell:Q', scale=alt.Scale(type='log')),
            x=alt.X('vol:Q', scale=alt.Scale(type='log')),
            color='dataset_name:N'
)

alt.Chart(...)

In [42]:
data

,gene_name,mass_frac,condition,strain,growth_rate_hr,dataset_name,go_terms,cog_class,cog_letter,fg_protein_per_cell,fg_per_cell
0,aas,0.000061,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016874; GO:0016746; GO:0016740; GO:0016020...,metabolism,Q,328.210357,0.020061
1,accA,0.000429,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016743; GO:0016740; GO:0042802; GO:0009317...,metabolism,I,328.210357,0.140839
2,accB,0.000578,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0009305; GO:0005829; GO:0005515; GO:0006631...,metabolism,H,328.210357,0.189561
3,accC,0.000782,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0005515; GO:0016874; GO:0006631; GO:0006629...,metabolism,I,328.210357,0.256620
4,accD,0.000496,0.4% sodium acetate,NCM3722,0.50107,Mori et al. 2021,GO:0016743; GO:0016740; GO:0009317; GO:0006631...,metabolism,I,328.210357,0.162710
...,...,...,...,...,...,...,...,...,...,...,...
146294,zapA,0.000061,glucose_minimal,MG1655,0.49000,Valgepea et al. 2013,GO:0090529; GO:0005886; GO:0005829; GO:0032153...,cellular processes and signaling,D,324.798702,0.019898
146295,zapB,0.000530,glucose_minimal,MG1655,0.49000,Valgepea et al. 2013,GO:0000917; GO:0032153; GO:0090529; GO:0036214...,cellular processes and signaling,D,324.798702,0.172181
146296,zipA,0.000211,glucose_minimal,MG1655,0.49000,Valgepea et al. 2013,GO:0043093; GO:0090529; GO:0032153; GO:0016021...,cellular processes and signaling,D,324.798702,0.068608
146297,zur,0.000022,glucose_minimal,MG1655,0.49000,Valgepea et al. 2013,GO:1900376; GO:0045892; GO:0044212; GO:0008270...,metabolism,P,324.798702,0.007243
